<a href="https://colab.research.google.com/github/NagarjunaD024/Datascience-LLMS/blob/main/src/Cost_optimizing_GPT/fine_tune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import openai
import pandas as pd
import time

from google.colab import drive, userdata,  output
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
client = openai.OpenAI(api_key = userdata.get('OPENAI_API_KEY'))

In [ ]:
"""
in_path = '/content/drive/My Drive/Colab Notebooks/DataScience+GPT/Data/textanalysis/train_reviews.jsonl'

reply = client.files.create(
  file=open(in_path, 'rb'), purpose='fine-tune')
file_id = reply.id

reply = client.fine_tuning.jobs.create(
  training_file=file_id, model='gpt-3.5-turbo')
job_id = reply.id
print(f'Job ID: {job_id}')

status = None
start_s = time.time()

while not (status == 'succeeded'):

  time.sleep(5)
  total_s = time.time() - start_s
  print(f'Fine-tuning since {total_s} seconds.')

  reply = client.fine_tuning.jobs.retrieve(job_id)
  status = reply.status
  print(f'Status: {status}')

print(f'Fine-tuning is finished!')

model_id = reply.fine_tuned_model
print(f'Model ID: {model_id}')
"""

"\nin_path = '/content/drive/My Drive/Colab Notebooks/DataScience+GPT/Data/textanalysis/train_reviews.jsonl'\n\nreply = client.files.create(\n  file=open(in_path, 'rb'), purpose='fine-tune')\nfile_id = reply.id\n\nreply = client.fine_tuning.jobs.create(\n  training_file=file_id, model='gpt-3.5-turbo')\njob_id = reply.id\nprint(f'Job ID: {job_id}')\n\nstatus = None\nstart_s = time.time()\n\nwhile not (status == 'succeeded'):\n  \n  time.sleep(5)\n  total_s = time.time() - start_s\n  print(f'Fine-tuning since {total_s} seconds.')\n  \n  reply = client.fine_tuning.jobs.retrieve(job_id)\n  status = reply.status\n  print(f'Status: {status}')\n    \nprint(f'Fine-tuning is finished!')\n\nmodel_id = reply.fine_tuned_model\nprint(f'Model ID: {model_id}')\n"

In [ ]:
job_info = client.fine_tuning.jobs.retrieve("ftjob-CiKcX24PkDB2TyoKm1BkvyVJ")
print(job_info)

FineTuningJob(id='ftjob-CiKcX24PkDB2TyoKm1BkvyVJ', created_at=1751443605, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-3.5-turbo-0125:personal::BoowtouP', finished_at=1751451945, hyperparameters=Hyperparameters(batch_size=1, learning_rate_multiplier=2.0, n_epochs=3), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-NAI7mh36xjkp9eGNfbFkxQtA', result_files=['file-Fh6LDhpXGgqTZwC9eUKqrn'], seed=1287908361, status='succeeded', trained_tokens=91170, training_file='file-W5FWJU2hgRoF3V7tY1o7ga', validation_file=None, estimated_finish=None, integrations=[], metadata=None, method=Method(type='supervised', dpo=None, reinforcement=None, supervised=SupervisedMethod(hyperparameters=SupervisedHyperparameters(batch_size=1, learning_rate_multiplier=2.0, n_epochs=3))), user_provided_suffix=None, usage_metrics=None, shared_with_openai=False, eval_id=None)


In [ ]:
def create_prompt(text):
  """ Create prompt for sentiment classification.
  Args:
  text: text to classify.
  Returns:
  Prompt for text classification.
  """
  return text

In [ ]:
def call_llm(prompt):
    """ Query large language model and return answer.

    Args:
        prompt: input prompt for language model.

    Returns:
        Answer by language model and total number of tokens.
    """
    for nr_retries in range(1, 4):
        try:
            response = client.chat.completions.create(
                model= "ft:gpt-3.5-turbo-0125:personal::BoowtouP",
                messages=[
                    {'role':'user', 'content':prompt}
                    ],
                temperature=0
                )

            answer = response.choices[0].message.content
            nr_tokens = response.usage.total_tokens
            return answer, nr_tokens

        except Exception as e:
            print(f'Exception: {e}')
            time.sleep(nr_retries * 2)

    raise Exception('Cannot query OpenAI model!')

In [ ]:
file_path = '/content/drive/My Drive/Colab Notebooks/DataScience+GPT/Data/textanalysis/reviews.csv'

df = pd.read_csv(file_path)

nr_correct = 0
nr_tokens = 0

for _, row in df.iterrows():

    text = row['text']
    prompt = create_prompt(text)
    label, current_tokens = call_llm(prompt)

    ground_truth = row['sentiment']
    if label == ground_truth:
        nr_correct += 1
    nr_tokens += current_tokens

    print(f'Label: {label}; Ground truth: {ground_truth}')

print(f'Number of correct labels:\t{nr_correct}')
print(f'Number of tokens used   :\t{nr_tokens}')

Label: neg; Ground truth: neg
Label: neg; Ground truth: neg
Label: neg; Ground truth: neg
Label: neg; Ground truth: neg
Label: pos; Ground truth: pos
Label: pos; Ground truth: neg
Label: pos; Ground truth: neg
Label: neg; Ground truth: neg
Label: neg; Ground truth: pos
Label: neg; Ground truth: neg
Number of correct labels:	7
Number of tokens used   :	1797
